<a href="https://www.kaggle.com/code/ayfait/cmc-gainers-notebook-starting-07-10-24?scriptVersionId=187942248" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### I wrote this first code on Kaggle to create a program that will compile a list of **Coinmarketcap daily gainers**.
I already wrote a code for the Top10 gainers treemap [here](https://github.com/AyFait/Cmc_Scrape_Top10_HeatMap_TreeMap)

In [1]:
#installing chrome browser
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:2 https://packages.cloud.google.com/apt gcsfuse-focal InRelease [1225 B]   
Get:3 https://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]  
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease   
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3776 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3745 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1245 kB]
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease               
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [3895 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4245 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1530 kB]
Fetched 18.7 MB in 2s (8457 kB/s) 

In [2]:
#installing chromedriver
!wget https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver


--2024-07-12 08:43:17--  https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 172.217.218.207, 142.250.145.207, 173.194.79.207, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|172.217.218.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7315061 (7.0M) [application/zip]
Saving to: 'chromedriver_linux64.zip'

chromedriver_linux6 100%[===================>]   6.98M  9.14MB/s    in 0.8s    

2024-07-12 08:43:18 (9.14 MB/s) - 'chromedriver_linux64.zip' saved [7315061/7315061]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
  inflating: LICENSE.chromedriver    


In [3]:
#insatlling selenium
!pip install --upgrade jupyter-lsp
!pip install --upgrade selenium
!pip install webdriver-manager


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: jupyter-lsp
    Found existing installation: jupyter-lsp 1.5.1
    Uninstalling jupyter-lsp-1.5.1:
      Successfully uninstalled jupyter-lsp-1.5.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 28.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.7.0
    Uninstalling websocket-client-1.7.0:
      Successfully uninstalled websocket-client-1.7.0


In [4]:
import os
import json
from kaggle_secrets import UserSecretsClient

import pandas as pd
import time
from datetime import datetime
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode.
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")  # This line helps in some environments
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


# Test the WebDrive
url='https://coinmarketcap.com/gainers-losers/'
driver.get(url)
print(driver.title)


See The Top Crypto Gainers And Losers Today [Updated] | CoinMarketCap


In [6]:
#define url of page to exract data from
url='https://coinmarketcap.com/gainers-losers/'
driver.get(url)
time.sleep(5) #Sleep for few seconds so, by that time, the webpage gets loaded.
ranking =  driver.find_elements(By.XPATH, '//*[@id="__next"]/div[2]/div/div[2]/div/div[2]/div/div[1]/div/table')# get element by XPATH from element selection in inspect mode


In [7]:
data = []  # Extract data
for index, rank in enumerate(ranking):
    rowData = rank.text.split('\n')
    #print(rowData) #This prints the extracted data all in a single row
    data.append(rowData)


In [8]:
#Resahaping the data list to # cols
lst_cleaned = rowData[5:] #popping headers
expectedRows = len(lst_cleaned) // 4 #divide the list by no of expected cols to get no of expected rows
lst_array = np.array(lst_cleaned)#turning the simngle rowdata list into an array
reshpd = lst_array.reshape(int(expectedRows), 4) #to get row x col
reshpdtrimmed = reshpd[:, :-1]#'Price' '24h%' 'Vol(24h)' were merged together so I popped them
forth = np.array([row[3].split() for row in reshpd])#now splitting 'Price' '24h%' 'Vol(24h)' on their own
lst_n_forth = np.concatenate((reshpdtrimmed, forth), axis = 1)#joining them back together

In [9]:
expectedRows = len(lst_cleaned) // 4 #divide the list by no of expected cols to get no of expected rows
lst_array = np.array(lst_cleaned)#turning the simngle rowdata list into an array
reshpd = lst_array.reshape(int(expectedRows), 4) 

In [10]:
#Adding TimeStamp for each row
timing = []
for i in range(len(lst_n_forth)):
    timestamp = datetime.utcnow().strftime('%m/%d/%Y %H:%M:%S UTC')
    timing.append(timestamp)
#print(timing)
timing_array = np.array(timing)    
timingreshpd = timing_array.reshape(len(timing), 1)
timed_array = np.concatenate((timingreshpd, lst_n_forth), axis = 1)#coming together making the perfect array with timestamp
#print(len(timed_array))
#print(timed_array)

In [11]:
#passing to nympy Dataframe to get tabular form
new_ver_csv = pd.DataFrame(timed_array,index=np.arange(1, len(timed_array)+1), columns=['Time_Stamp', 'CMC_Rank', 'Name', 'Symbol', 'Price($)', '24h%', '24h_Vol($)'])
new_ver_csv['Price($)'] = new_ver_csv['Price($)'].str.lstrip('$').str.replace(',', '').astype(float)
new_ver_csv['24h%'] = new_ver_csv['24h%'].str.rstrip('%').str.replace(',', '').astype(float)
new_ver_csv['24h_Vol($)'] = new_ver_csv['24h_Vol($)'].str.lstrip('$').str.replace(',', '').astype(float)
#print(df)
timestamp = datetime.utcnow().strftime('%b %d, %Y %H:%M:%S UTC')
title = f"CMC 24h% Increase for {timestamp}"
display_title = f"**{title}**"
# Display the title and DataFrame as a Markdown
from IPython.display import display, Markdown
display(Markdown(display_title))
display(new_ver_csv)


**CMC 24h% Increase for Jul 12, 2024 08:44:26 UTC**

,Time_Stamp,CMC_Rank,Name,Symbol,Price($),24h%,24h_Vol($)
1,07/12/2024 08:44:26 UTC,72,MANTRA,OM,1.01000,5.59,2.010981e+08
2,07/12/2024 08:44:26 UTC,38,Maker,MKR,2400.09000,5.15,1.033788e+08
3,07/12/2024 08:44:26 UTC,55,Aave,AAVE,94.30000,4.21,1.259542e+08
4,07/12/2024 08:44:26 UTC,7,XRP,XRP,0.46340,4.20,1.295956e+09
5,07/12/2024 08:44:26 UTC,30,Stellar,XLM,0.09095,2.99,5.709579e+07
6,07/12/2024 08:44:26 UTC,19,NEAR Protocol,NEAR,4.82000,2.72,3.538118e+08
7,07/12/2024 08:44:26 UTC,94,Safe,SAFE,1.49000,2.66,1.548835e+07
8,07/12/2024 08:44:26 UTC,16,Bitcoin Cash,BCH,349.95000,2.17,2.842393e+08
9,07/12/2024 08:44:26 UTC,8,Toncoin,TON,7.32000,1.92,2.589467e+08
10,07/12/2024 08:44:26 UTC,17,UNUS SED LEO,LEO,5.79000,1.92,2.898997e+06


In [12]:
os.makedirs('/kaggle/dataset/', exist_ok=True)

#First load exixting csv from input
old_ver_csv_path = '/kaggle/input/cmc-gain-dataset/CMC_24h_Gainers_Starting_07_10_24 updated.csv'
#new_csv_file_path = f'/kaggle/dataset/CMC_24h_Gainers_Starting_07_10_24 updated.csv'
new_csv_file_path = f'/kaggle/dataset/CMC_24h_Gainers_Starting_07_10_24 updated.csv'


if not os.path.isfile(old_ver_csv_path):
    new_ver_csv.to_csv(new_csv_file_path)
    
else:
    #Load the older version of the csv file
    old_ver_csv = pd.read_csv(old_ver_csv_path)
    #remove the existing index of the old csv file
    old_ver_csv = old_ver_csv.iloc[:, 1:]
    #Join old and new csv files
    joined = pd.concat([old_ver_csv, new_ver_csv], axis=0)
    #This line keeps creating new index on each run
    joined.index = range(1, len(joined)+1)
    #joined.reset_index(drop=True, inplace=True)

    
display(joined)


,Time_Stamp,CMC_Rank,Name,Symbol,Price($),24h%,24h_Vol($)
1,07/11/2024 11:57:31 UTC,32,Stacks,STX,1.6800,12.57,168190316.0
2,07/11/2024 11:57:31 UTC,96,zkSync,ZK,0.1700,6.43,137707042.0
3,07/11/2024 11:57:31 UTC,44,Optimism,OP,1.7000,5.57,169800859.0
4,07/11/2024 11:57:31 UTC,56,Ondo,ONDO,1.0200,5.41,131065186.0
5,07/11/2024 11:57:31 UTC,53,Lido DAO,LDO,1.6400,5.13,114716407.0
...,...,...,...,...,...,...,...
80,07/12/2024 08:44:26 UTC,27,Monero,XMR,158.9800,0.34,70786292.0
81,07/12/2024 08:44:26 UTC,97,Conflux,CFX,0.1416,0.30,27374931.0
82,07/12/2024 08:44:26 UTC,83,Tezos,XTZ,0.7498,0.26,20217545.0
83,07/12/2024 08:44:26 UTC,40,OKB,OKB,36.6200,0.10,2837952.0


In [13]:

# Check if the file exists in dataset folder
if not os.path.isfile(new_csv_file_path):
    #Create new
    joined.to_csv(new_csv_file_path)#with index
    #joined.to_csv(new_csv_file_path, index=False)#without index
else:
    #Append to existing     
    joined.to_csv(new_csv_file_path, mode='a', header=False, index=False)

In [14]:
#Setting up environment variables in order to use the kaggle CLI to automate upoloading of the dataset

secrets = UserSecretsClient()

os.environ['KAGGLE_USERNAME'] = secrets.get_secret("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = secrets.get_secret("KAGGLE_KEY")

In [15]:
#From the folder that holds the dataset and creating the metadata file, It must NOT be the working dir else Unncessary files will be uploaded too:

meta = dict(
    id="ayfait/CMC-Gain-Dataset",
    title="CMC Gainers Dataset Starting_07_11_2024",
    isPrivate=False,
    licenses=[dict(name="apache-2.0")]
)

with open('/kaggle/dataset/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)

In [16]:
#To upload the dataset created inside /kaggle/dataset for the first time:
#!kaggle datasets create -p "/kaggle/dataset" --dir-mode zip

#To push a new version of the dataset 
!kaggle datasets version -p "/kaggle/dataset" -m "Updated via notebook" --dir-mode zip

Starting upload for file CMC_24h_Gainers_Starting_07_10_24 updated.csv
100%|██████████████████████████████████████| 5.38k/5.38k [00:00<00:00, 16.8kB/s]
Upload successful: CMC_24h_Gainers_Starting_07_10_24 updated.csv (5KB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayfait/CMC-Gain-Dataset


In [17]:
#To clean directory
#rm -rf /kaggle/working/*
